<a href="https://colab.research.google.com/github/Eleniy1/LDA_Icelandic/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_www_hugi_is.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для www.hugi.is

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import re

Перебор страниц

In [ ]:
main_link = []
meta = []
n = 0
while n<342:
  n+=1
  link1 = 'https://www.hugi.is/ljod/greinar/heitast/?sida='+str(n)
  main_link.append(link1)
print(main_link)


['https://www.hugi.is/ljod/greinar/heitast/?sida=1', 'https://www.hugi.is/ljod/greinar/heitast/?sida=2', 'https://www.hugi.is/ljod/greinar/heitast/?sida=3', 'https://www.hugi.is/ljod/greinar/heitast/?sida=4', 'https://www.hugi.is/ljod/greinar/heitast/?sida=5', 'https://www.hugi.is/ljod/greinar/heitast/?sida=6', 'https://www.hugi.is/ljod/greinar/heitast/?sida=7', 'https://www.hugi.is/ljod/greinar/heitast/?sida=8', 'https://www.hugi.is/ljod/greinar/heitast/?sida=9', 'https://www.hugi.is/ljod/greinar/heitast/?sida=10', 'https://www.hugi.is/ljod/greinar/heitast/?sida=11', 'https://www.hugi.is/ljod/greinar/heitast/?sida=12', 'https://www.hugi.is/ljod/greinar/heitast/?sida=13', 'https://www.hugi.is/ljod/greinar/heitast/?sida=14', 'https://www.hugi.is/ljod/greinar/heitast/?sida=15', 'https://www.hugi.is/ljod/greinar/heitast/?sida=16', 'https://www.hugi.is/ljod/greinar/heitast/?sida=17', 'https://www.hugi.is/ljod/greinar/heitast/?sida=18', 'https://www.hugi.is/ljod/greinar/heitast/?sida=19', '

In [ ]:
base_url = 'https://www.hugi.is'
texts = pd.DataFrame({'poem_link': [], 'author_link': [], 'text': [], 'poem_date': []})
for circle1 in main_link[:]:
  soup1 = BeautifulSoup(urllib.request.urlopen(circle1))
  for soup2 in soup1.find_all('tr', {'class': 'item'}):
    poem_link = base_url + soup2.find('td', {'class': 'titlecol'})('a')[0]['href']
    author_link = base_url + '/unknown/'
    try:
        author_link = base_url + soup2.find('td', {'class': 'authorcol'})('a')[0]['href']
    except:
        pass
    poem_date = soup2.find('td', {'class': 'createdcol'}).find('span').text[:12]

    soup3 = BeautifulSoup(urllib.request.urlopen(poem_link))
    text = soup3.find('div', class_='articlebody').text.strip().replace('\r', '').replace('\n', ' ').replace('\t', ' ').split()
    text = ' '.join(text)
        
    texts = texts.append(pd.Series({'poem_link': poem_link, 'author_link': author_link, 'text': text, 'poem_date': poem_date}), ignore_index=True)    

In [ ]:
texts.head(3)

,poem_link,author_link,text,poem_date
0,https://www.hugi.is/ljod/greinar/803008/vinar-...,https://www.hugi.is/notendur/gufa/,"orð vitundar, vinar lykill Vinar lykill Vegur ...",28. jún 2016
1,https://www.hugi.is/ljod/greinar/802980/desemb...,https://www.hugi.is/notendur/gufa/,Desember dimman dregur mig inn í ljósin og yl ...,15. des 2015
2,https://www.hugi.is/ljod/greinar/802945/lettfa...,https://www.hugi.is/notendur/gufa/,Léttfættur í stígvélunum gengur til vinnu sérh...,30. maí 2015


In [ ]:
texts[texts.author_link.str.endswith('known/')].head()

,poem_link,author_link,text,poem_date
57,https://www.hugi.is/ljod/greinar/727213/einhva...,https://www.hugi.is/unknown/,Fyrrverandi sem ég kynntist á geðspítala..Anti...,17. júl 2011
58,https://www.hugi.is/ljod/greinar/727212/duck-b...,https://www.hugi.is/unknown/,Tjaah.. Ég spurði hann hvað ég ætti að skrifa ...,16. júl 2011
102,https://www.hugi.is/ljod/greinar/708639/saeluf...,https://www.hugi.is/notendur/unknown/,Í gegnum mínar eigin stunurheyri ég hvalarhljó...,10. sep 2010
103,https://www.hugi.is/ljod/greinar/708486/onefnt/,https://www.hugi.is/unknown/,Máninn tilbiður komu þínaMig langar þér að sýn...,8. sep 2010
114,https://www.hugi.is/ljod/greinar/696129/hungur...,https://www.hugi.is/unknown/,Hungur verkamannsins Frostið bítur bitran mann...,31. mar 2010


In [ ]:
authors = pd.DataFrame({'author_link': [base_url + '/unknown/'], 'age': [None], 'sex': [None]})

for author_link in texts.author_link.unique():
    try:
        soup4 = BeautifulSoup(urllib.request.urlopen(author_link))
    except:
        continue
        
    meta = soup4.find('div', class_='usermeta').text.strip().split('\n')
    age, sex = None, None
    for s in meta:
        r = re.search(r'([0-9]+)\sára\s(kvenmaður|karlmaður)', s)
        if r:
            age, sex = r[1], r[2]
    authors = authors.append({'author_link': author_link, 'age': age, 'sex': sex}, ignore_index=True)

In [ ]:
authors.head()

,author_link,age,sex
0,https://www.hugi.is/unknown/,None,None
1,https://www.hugi.is/notendur/gufa/,None,None
2,https://www.hugi.is/notendur/pardus/,39,karlmaður
3,https://www.hugi.is/notendur/Gumma/,None,None
4,https://www.hugi.is/notendur/Raggatheo/,34,kvenmaður


In [ ]:
texts1 = texts.merge(authors, how='inner', left_on='author_link', right_on='author_link')[['text', 'age', 'sex', 'poem_date']]
texts1.head(10)

,text,age,sex,poem_date
0,"orð vitundar, vinar lykill Vinar lykill Vegur ...",None,None,28. jún 2016
1,Desember dimman dregur mig inn í ljósin og yl ...,None,None,15. des 2015
2,Léttfættur í stígvélunum gengur til vinnu sérh...,None,None,30. maí 2015
3,Hugprúða frostrós fannhvíti hugur. Fimm lítil ...,None,None,6. mar 2015
4,Flaugst í hjartað mitt litli rauðbrystingur. B...,None,None,3. jan 2015
5,Dropar flétta saman frið Um rifu augans Orka Í...,None,None,22. mar 2013
6,Kærleiks höndin bjarta haltu mér um stund Í fj...,None,None,17. júl 2012
7,"Ég er frjáls undir flöktandi stjörnu, feiminn ...",None,None,8. maí 2012
8,Ljósir lokkar prýðalifandi elda.Á hvítum tindi...,None,None,12. feb 2012
9,Sveimandi horfaleika sérSvífandi sindrandi ljó...,None,None,6. des 2011


In [ ]:
texts1[texts1.text.str.startswith('Fyrrverandi sem')].head()

,text,age,sex,poem_date
799,Fyrrverandi sem ég kynntist á geðspítala..Anti...,None,None,17. júl 2011


In [ ]:
texts1.to_csv('hugi.is.poems.csv', index=False)